In [ ]:
#####################################################################################
# Audio-driven upper-body motion synthesis on a humanoid robot
# Computer Science Tripos Part III Project
# Jan Ondras (jo356@cam.ac.uk), Trinity College, University of Cambridge
# 2017/18
#####################################################################################
# Simulation of 3D pose reconstructed by the 3D pose estimation method OP+M
#####################################################################################

In [ ]:
# OpenPose + Matching

In [1]:
#############################################################
# Convert Openpose 2D joints to .mat matrices; then run MATLAB to get 3D
#############################################################

import numpy as np
import matplotlib.pyplot as plt
from scipy.io import savemat
import json
import os
import glob

# openpose_COCO_joints = 
# 	            {0,  "Nose"},
#         {1,  "Neck"},
#         {2,  "RShoulder"},
#         {3,  "RElbow"},
#         {4,  "RWrist"},
#         {5,  "LShoulder"},
#         {6,  "LElbow"},
#         {7,  "LWrist"},
#         {8,  "RHip"},
#         {9,  "RKnee"},
#         {10, "RAnkle"},
#         {11, "LHip"},
#         {12, "LKnee"},
#         {13, "LAnkle"},
#            	{14, "REye"},
#           	{15, "LEye"},
#           	{16, "REar"},
#           	{17, "LEar"},
#           	{18, "Background"}
                        
# for this VID
VID = 'PID20Task2'
VID = 'PID23Task3'
# os.mkdir('./../Dataset/OpenposePoseFeatures_mat/' + VID)

selected_joints = [0,1,2,3,4,5,6,7,8,9,10,11,12,13] # 14 from COCO model
# To match their (3D from 2D) model 14
# % (head)
# % (neck)
# % (right shoulder)
# % (right elbow)
# % (right wrist)
# % (left shoulder)
# % (left elbow)
# % (left wrist)
# % (right hip)
# % (right knee)
# % (right ankle)
# % (left hip)
# % (left knee)
# % (left ankle)

json_filenames = glob.glob('./../Dataset/OpenposePoseFeatures/' + VID + '/*.json')
print( 'VID:', VID, ',\t #frames: ', len(json_filenames)) # 1302

# Iterate over all frames: IN ANY ORDER!!! doesn't matter here
for json_filename in json_filenames: 
    data = json.load(open(json_filename))
    if len(data['people']) != 1:
        print ("Error: multiple people detected, in ", json_filename)
                       
    f = np.reshape(data['people'][0]['pose_keypoints'], (-1,3)) 
    f = f[selected_joints] # keep only 14 joints
    f = f[:, [0,1]] # only x, y, omit score
    #print (f)
                       
    savemat('./../Dataset/OpenposePoseFeatures_mat/' + VID + '/' + json_filename.split('/')[-1][:-5], 
           {'prediction': f})


VID: PID23Task3 ,	 #frames:  2516


In [9]:
#########################################################################################################
# Visualize 3D joint positions using 
# "3D Human Pose Estimation = 2D Pose Estimation + Matching"
# 2D pose from Openpose
# COCO  model
#########################################################################################################

from geoutils import radToDeg, xyz_to_angles
from vpython import *
from scipy.io import loadmat
import numpy as np
from geo import *


VID = 'PID20Task2'
VID = 'PID23Task3'

STOP_AT_FRAMES = [1360, 1375, 1390]
# STOP_AT_FRAMES = [1375, 1390]
# STOP_AT_FRAMES = [1390]

show_labels = True
show_labels = False

smooth_joints_pos = True
smooth_joints_pos = False

FR = 1000

#################################################
showRobot = True # simultaneously send commands to the robot
showRobot = False

angles_names = [
    "HeadPitch", "HeadYaw", 
    "LShoulderRoll", "LShoulderPitch", "LElbowRoll", "LElbowYaw",
    "RShoulderRoll", "RShoulderPitch", "RElbowRoll", "RElbowYaw", 
    "HipRoll", "HipPitch"
]
if showRobot:
    print ("Showing on virtual robot ...")
    from naoqi import ALProxy
    IP = "127.0.0.1"
    port = 45637
    motionProxy = ALProxy("ALMotion", IP, port)
    # Reset robot to neutral pose
    for an in angles_names:
        angle_reset = 0.
        if an == 'LShoulderPitch' or an == 'RShoulderPitch':
            angle_reset = angle_reset + np.pi/2
        motionProxy.setAngles(an, angle_reset, 1.)
        
    jointID = {     # Mapping: joint names -> position in array as extracted
        #'Pelvis'    : NA,  # not available => have to estimate from hips
        'RHip'      : 8,    
        'LHip'      : 11,   
        #'Spine'     : NA,  # not available, not used
        'Neck'      : 1,
        #'Nose'      : NA,  # not available => have to estimate HeadYaw another way
        'HeadTop'   : 0,  
        'LShoulder' : 5,
        'LElbow'    : 6,
        'LWrist'    : 7,
        'RShoulder' : 2,
        'RElbow'    : 3,
        'RWrist'    : 4
    }
#################################################

# joints_used = np.arange(17) # first 17
joints_used = [0,1,2,3,4,5,6,7, 8, 11] # 14 from COCO model, w/o legs

joints_names = ['head_top','neck','right_shoulder','right_elbow','right_wrist',
                'left_shoulder','left_elbow','left_wrist',
                'right_hip','right_knee','right_ankle',
                'left_hip','left_knee','left_ankle']
joint_connections = [
    (0, 1),                                       # central, blue
    (1,8), (1,11),                                # extra for Openpose
    (1, 2), (2, 3), (3, 4), (8, 9), (9, 10),      # right, red
    (1, 5), (5, 6), (6, 7), (11, 12), (12, 13),   # left, green
]
joint_connections_colors = [color.blue]*3 + [color.red]*5 + [color.green]*5
joints_colors = [color.white]*len(joints_names)

if smooth_joints_pos:
    data = np.load('./../Dataset/Openpose_3D_smoothed/' + VID + '.npz')['d']
else:
    data = loadmat('./../Dataset/Openpose_3D/' + VID + '.mat')['M']
    data = np.reshape(data, (-1,14,3))
print ("Shape =" , np.shape(data))

# Setup scene
scene = canvas()
scene.width = 960
scene.height = 600
scene.title = "Openpose 3D pose reconstruction"
scene.scale = 400.
# scene.fov = np.pi /3.
scene.up=vector(0.,-1.,0.)     # To flip scene horiznotally !!!
scene.forward=vector(0.,0.,1.)     # To rotate around y !!!
# scene.center=vector(0.,-150.,10000) # To shift down a but

###################################### Centering to neck joint
data[0, :, :] = data[0, :, :] - data[0, 1, :]
# data[:,:,2] = 0. # if you remove depth

scene.lights = []
scene.ambient=color.gray(0.8)

# Texture uderneath
y_plane_pos = 500
plane_size = 1000
# a = vertex( pos=vector(-plane_size, y_plane_pos, -plane_size) )
# b = vertex( pos=vector(-plane_size, y_plane_pos,  plane_size) )
# c = vertex( pos=vector( plane_size, y_plane_pos,  plane_size) )
# d = vertex( pos=vector( plane_size, y_plane_pos, -plane_size) )
# quad(vs=[a,b,c,d], texture=textures.rug)
# SHOW PLANE UNDERNEATH THE FIGURE
#box(pos=vec(0,y_plane_pos,0), length=plane_size, height=1, width=plane_size, texture=textures.rough)

frame = 0
scene.caption = 'Frame: ' + str(frame)
Ps = []
Es = []
Ls = []
# Draw all joints requested
for i in joints_used:
    x = data[frame, i, 0] #- skeleton_center[0]
    y = data[frame, i, 1] #- skeleton_center[1]
    z = data[frame, i, 2] #- skeleton_center[2]
    P = sphere(pos=vector(x,y,z), radius=10, color=joints_colors[i])#, #size_units="world")
    Ps.append(P)
    if show_labels:
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=20,
                yoffset=50, space=30,
                height=16, border=3,
                font='sans')
        Ls.append(L)

# Draw all links between joints
for i, (a,b) in enumerate(joint_connections):
    if a in joints_used and b in joints_used:
        ax = data[frame, a, 0] #- skeleton_center[0]
        ay = data[frame, a, 1] #- skeleton_center[1]
        az = data[frame, a, 2] #- skeleton_center[2]
        bx = data[frame, b, 0] #- skeleton_center[0]
        by = data[frame, b, 1] #- skeleton_center[1]
        bz = data[frame, b, 2] #- skeleton_center[2]
        cx = (ax + bx) / 2.
        cy = (ay + by) / 2.
        cz = (az + bz) / 2.
        E = ellipsoid(pos=vector(cx,cy,cz), axis=vector(ax-bx,ay-by,az-bz), length=np.linalg.norm([ax-bx,ay-by,az-bz]), 
                  height=25, width=25, color=joint_connections_colors[i])
        Es.append(E)


# Animate
for frame in range(1, len(data)):
    
    ####################################################################
    # Calculate joint angles & Send commands to the robot
    if showRobot:
        xyz = data[frame]
        #xyz[:,2] = - xyz[:,2]
        ####################################################################
        # Calculate 12 joint angles, if not possible for this method set NAN (np.nan)
        # xyz[jointID['Pelvis']] gives xyz coordinates of Pelvis joint
        # For OP+M, pass None for Nose position => can only estimate HeadYaw
        # And estimate Pelvis position as midpoint of LHip-RHip
        PE = Point(xyz[jointID['LHip']]).midpoint_to(Point(xyz[jointID['RHip']]))

        jas = xyz_to_angles(Point(xyz[jointID['HeadTop']]), None, 
                            Point(xyz[jointID['Neck']]),    PE, 
                    Point(xyz[jointID['LShoulder']]), Point(xyz[jointID['LElbow']]), Point(xyz[jointID['LWrist']]), 
                    Point(xyz[jointID['RShoulder']]), Point(xyz[jointID['RElbow']]), Point(xyz[jointID['RWrist']])                  
        )
        
        for ang_ind, ang in enumerate(jas):
            if not np.isnan(ang): # take only angles available for this method
                motionProxy.setAngles(angles_names[ang_ind], ang, 1.) 
    
    ###################################### Centering to neck joint
    data[frame, :, :] = data[frame, :, :] - data[frame, 1, :]
    
    rate(FR)
    scene.caption = 'Frame: ' + str(frame)
    # Draw all joints requested
    cnt = 0
    for i in joints_used:
        x = data[frame, i, 0] #- skeleton_center[0]
        y = data[frame, i, 1] #- skeleton_center[1]
        z = data[frame, i, 2] #- skeleton_center[2]
        Ps[cnt].pos = vector(x,y,z)
        if show_labels:
            Ls[cnt].pos = vector(x,y,z)
        cnt += 1

    # Draw all links between joints
    cnt = 0
    for i, (a,b) in enumerate(joint_connections):
        if a in joints_used and b in joints_used:
            ax = data[frame, a, 0] #- skeleton_center[0]
            ay = data[frame, a, 1] #- skeleton_center[1]
            az = data[frame, a, 2] #- skeleton_center[2]
            bx = data[frame, b, 0] #- skeleton_center[0]
            by = data[frame, b, 1] #- skeleton_center[1]
            bz = data[frame, b, 2] #- skeleton_center[2]    
            cx = (ax + bx) / 2.
            cy = (ay + by) / 2.
            cz = (az + bz) / 2.
            Es[cnt].pos = vector(cx,cy,cz)
            Es[cnt].axis = vector(ax-bx,ay-by,az-bz)
            Es[cnt].length = np.linalg.norm([ax-bx,ay-by,az-bz])
            cnt += 1
            
    if frame in STOP_AT_FRAMES:
        break

Shape = (2516, 14, 3)


<IPython.core.display.Javascript object>